In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install required libraries in Google Colab
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!pip install matplotlib opencv-python numpy pandas tqdm albumentations pillow tensorboard torchmetrics
!pip install torch torchvision tqdm

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninst

In [3]:
import torch
import torchvision
import numpy as np
import cv2
import albumentations as A
import matplotlib.pyplot as plt

print("All libraries imported successfully!")
print("GPU Available:", torch.cuda.is_available())


All libraries imported successfully!
GPU Available: True


In [5]:
# Step 1: Import Libraries
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models
from torch.utils.data import DataLoader
from torchvision.models import ResNet50_Weights
import os
from tqdm import tqdm  # Progress bar for training

# Step 2: Mount Google Drive (if using Google Colab)
from google.colab import drive
drive.mount('/content/drive')

# Step 3: Data Augmentation & Normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Step 4: Load Dataset from Google Drive
data_dir = "/content/drive/MyDrive/dataset"  # Change this if needed

print("✅ Dataset Found!")

train_data = datasets.ImageFolder(root=os.path.join(data_dir, "train"), transform=transform)
val_data = datasets.ImageFolder(root=os.path.join(data_dir, "val"), transform=transform)
test_data = datasets.ImageFolder(root=os.path.join(data_dir, "test"), transform=transform)

print(f"✅ Train: {len(train_data)} images, ✅ Val: {len(val_data)} images, ✅ Test: {len(test_data)} images")

# Step 5: Create Data Loaders
batch_size = 16  # Increase batch size if you have more VRAM
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

print("✅ Data Loaders Ready!")

# Step 6: Load Pretrained ResNet50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 2 classes (biodegradable, non-biodegradable)
model = model.to(device)

# Step 7: Loss Function & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Step 8: Training Loop
num_epochs = 10
best_loss = float('inf')

print("🚀 Training started...")
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix(loss=f"{loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"🔄 Epoch {epoch+1} Loss: {avg_loss:.4f}")

    # Save best model
    if avg_loss < best_loss:
        best_loss = avg_loss
        model_path = "/content/drive/MyDrive/resnet50_biodegradable_best.pt"
        torch.save(model.state_dict(), model_path)
        print("✅ Model saved!")

# Step 9: Evaluate on Test Set
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"🎯 Test Accuracy: {accuracy:.2f}%")

evaluate(model, test_loader)  # ✅ ADDED TESTING PHASE


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Dataset Found!
✅ Train: 2722 images, ✅ Val: 262 images, ✅ Test: 264 images
✅ Data Loaders Ready!
🚀 Training started...


Epoch 1/10: 100%|██████████| 171/171 [15:02<00:00,  5.28s/it, loss=0.1973]


🔄 Epoch 1 Loss: 0.2746
✅ Model saved!


Epoch 2/10: 100%|██████████| 171/171 [00:56<00:00,  3.03it/s, loss=0.5909]


🔄 Epoch 2 Loss: 0.1664
✅ Model saved!


...

🔄 Epoch 8 Loss: 0.1077


Epoch 9/10: 100%|██████████| 171/171 [00:55<00:00,  3.06it/s, loss=0.0044]


🔄 Epoch 9 Loss: 0.0534


Epoch 10/10: 100%|██████████| 171/171 [00:55<00:00,  3.09it/s, loss=0.4995]


🔄 Epoch 10 Loss: 0.0386
✅ Model saved!
🎯 Test Accuracy: 97.73%


In [4]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
from google.colab import drive

# Define device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = "/content/drive/MyDrive/resnet50_biodegradable_best.pt"  # Update path if needed
model = models.resnet50()
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 2)  # 2 classes: Biodegradable, Non-Biodegradable
model.load_state_dict(torch.load(model_path, map_location=device))
model = model.to(device)
model.eval()  # Set model to evaluation mode

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Prediction function
def predict_image(image_path, model):
    image = Image.open(image_path).convert("RGB")  # Open image
    image = transform(image).unsqueeze(0).to(device)  # Transform & move to device

    with torch.no_grad():
        outputs = model(image)  # Get predictions
        _, predicted = torch.max(outputs, 1)  # Get class with highest probability

    class_names = ["Biodegradable", "Non-Biodegradable"]
    print(f"Prediction: {class_names[predicted.item()]}")

# Test with a sample image
image_path = "/content/001.jpg"  # Update with your test image path
predict_image(image_path, model)

<ipython-input-4-60e9526313e4>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


Prediction: Biodegradable
